Loading Data 
1. Air Quality Data

In [ ]:
!pip install fiona
!pip install geopandas
!pip install folium
!pip install sodapy -q
!pip install adjustText
!pip install plotly --upgrade
!pip install contextily
!pip install mapclassify
!pip install chart-studio

     |████████████████████████████████| 15.4 MB 5.1 MB/s 
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 34.8 MB/s 
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7094 sha256=befe5570a34739c3a897e1756e40699c07eb41e5caaa9fde58a738932399530e
  Stored in directory: /root/.cache/pip/wheels/2f/98/32/afbf902d8f040fadfdf0a44357e4ab750afe165d873bf5893d
Successfully built adjustText
     |████████████████████████████████| 25.3 MB 58.7 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 19.3 MB 7.2 MB/s 
     |████████████████████████████████| 64 kB 1.7 MB/s 


In [ ]:
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
import numpy as np

# Load a required package 
# This is a library for accessing and parsing data through URLs
from urllib.parse import urlencode
import urllib.request, json 
from bs4 import BeautifulSoup # for web scraping
import fiona
import plotly.express as px
import geopandas as gpd
import adjustText as aT
import seaborn as sns # visualization styling package
import plotly.graph_objects as go
import urllib, json
import chart_studio.plotly as py
import chart_studio.tools as tls
import os
from sodapy import Socrata
from IPython.display import Image, display
from shapely.geometry import Polygon
import time
from shapely.geometry import shape
import contextily as ctx


# A magic functin that renders the figure in a notebook 
%matplotlib inline 

from matplotlib import pyplot as plt



In [ ]:
import folium
from folium.plugins import HeatMap

Getting NYC Air Quality Data from NYC Open Data Portal : https://data.cityofnewyork.us/Environment/Air-Quality/c3uy-2p5r

In [ ]:
API = 'https://data.cityofnewyork.us/resource/c3uy-2p5r.csv?'

In [ ]:
query = {'$select':'*',
         '$where': 'start_date > "2014-12-31"',
         '$order': 'unique_id',
         '$limit': 14500} #change to 414500 later

aq_df = pd.read_csv(API + urlencode(query))
aq_df.head()

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,message
0,179718,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,504,South Beach - Tottenville,2015,2015-01-01T00:00:00.000,2.0,NaN
1,179719,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,503,Willowbrook,2015,2015-01-01T00:00:00.000,2.1,NaN
2,179720,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,501,Port Richmond,2015,2015-01-01T00:00:00.000,2.8,NaN
3,179721,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,502,Stapleton - St. George,2015,2015-01-01T00:00:00.000,4.6,NaN
4,179722,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,410,Rockaways,2015,2015-01-01T00:00:00.000,6.1,NaN


In [ ]:
aq_df.dtypes

unique_id           int64
indicator_id        int64
name               object
measure            object
measure_info       object
geo_type_name      object
boro_cd             int64
geo_place_name     object
time_period        object
start_date         object
data_value        float64
message           float64
Date               object
Time               object
dtype: object

In [ ]:
aq_df['Date'] = pd.to_datetime(aq_df['start_date']).dt.date
aq_df['Time'] = pd.to_datetime(aq_df['start_date']).dt.time

In [ ]:
print(aq_df.shape[0])
aq_df.dropna()
print(aq_df.shape[0])

4752
4752


In [ ]:
aq_df.head()

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value,message,Date,Time
0,179718,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,504,South Beach - Tottenville,2015,2015-01-01T00:00:00.000,2.0,NaN,2015-01-01,00:00:00
1,179719,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,503,Willowbrook,2015,2015-01-01T00:00:00.000,2.1,NaN,2015-01-01,00:00:00
2,179720,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,501,Port Richmond,2015,2015-01-01T00:00:00.000,2.8,NaN,2015-01-01,00:00:00
3,179721,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,502,Stapleton - St. George,2015,2015-01-01T00:00:00.000,4.6,NaN,2015-01-01,00:00:00
4,179722,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,410,Rockaways,2015,2015-01-01T00:00:00.000,6.1,NaN,2015-01-01,00:00:00


In [ ]:
aq_df.dtypes

unique_id           int64
indicator_id        int64
name               object
measure            object
measure_info       object
geo_type_name      object
geo_join_id         int64
geo_place_name     object
time_period        object
start_date         object
data_value        float64
message           float64
Date               object
Time               object
dtype: object

In [ ]:
aq_df = aq_df.rename(columns={"geo_join_id":"boro_cd"})

In [ ]:
!git clone https://github.com/kirthi-b/QoL_NYC_Children/

Cloning into 'QoL_NYC_Children'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 108 (delta 30), reused 84 (delta 14), pack-reused 0
Receiving objects: 100% (108/108), 11.14 MiB | 14.80 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
gdf = gpd.read_file("/content/QoL_NYC_Children/Environmental/Data/Air_Quality/Community Districts.zip")

In [ ]:
gdf.head()

,boro_cd,shape_area,shape_leng,geometry
0,206.0,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843..."
1,404.0,6.573966e+07,37018.373718,"POLYGON ((-73.84751 40.73901, -73.84801 40.738..."
2,203.0,4.479687e+07,33500.069106,"POLYGON ((-73.88072 40.83752, -73.88074 40.837..."
3,304.0,5.666261e+07,37007.806599,"POLYGON ((-73.89647 40.68234, -73.89653 40.682..."
4,205.0,3.831698e+07,29443.048128,"POLYGON ((-73.89138 40.86170, -73.89142 40.861..."


In [ ]:
gdf.dtypes

boro_cd        float64
shape_area     float64
shape_leng     float64
geometry      geometry
dtype: object

In [ ]:
gdf = gdf.astype({"boro_cd":'int'})
gdf.head()

,boro_cd,shape_area,shape_leng,geometry
0,206,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843..."
1,404,6.573966e+07,37018.373718,"POLYGON ((-73.84751 40.73901, -73.84801 40.738..."
2,203,4.479687e+07,33500.069106,"POLYGON ((-73.88072 40.83752, -73.88074 40.837..."
3,304,5.666261e+07,37007.806599,"POLYGON ((-73.89647 40.68234, -73.89653 40.682..."
4,205,3.831698e+07,29443.048128,"POLYGON ((-73.89138 40.86170, -73.89142 40.861..."


In [ ]:
gdf_join = gdf.merge(aq_df, on="boro_cd", how='left')
gdf_join.head()


,boro_cd,shape_area,shape_leng,geometry,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_place_name,time_period,start_date,data_value,message,Date,Time
0,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179744.0,642.0,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,34.4,NaN,2015-01-01,00:00:00
1,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179792.0,640.0,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,1.1,NaN,2015-01-01,00:00:00
2,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179840.0,641.0,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,0.2,NaN,2015-01-01,00:00:00
3,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325103.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,Borough Park,2016,2016-01-01T00:00:00.000,0.6,NaN,2016-01-01,00:00:00
4,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325151.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,CD,Belmont and East Tremont (CD6),2016,2016-01-01T00:00:00.000,3.8,NaN,2016-01-01,00:00:00


In [ ]:
gdf_join.drop_duplicates()
gdf_join.drop(columns=['message','start_date'])

,boro_cd,shape_area,shape_leng,geometry,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_place_name,time_period,data_value,Date,Time
0,206,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179744.0,642.0,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,Borough Park,2015,34.40,2015-01-01,00:00:00
1,206,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179792.0,640.0,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,Borough Park,2015,1.10,2015-01-01,00:00:00
2,206,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179840.0,641.0,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,Borough Park,2015,0.20,2015-01-01,00:00:00
3,206,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325103.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,Borough Park,2016,0.60,2016-01-01,00:00:00
4,206,4.266431e+07,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325151.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,CD,Belmont and East Tremont (CD6),2016,3.80,2016-01-01,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4269,209,1.142849e+08,62243.281229,"POLYGON ((-73.83979 40.83562, -73.83967 40.835...",605826.0,365.0,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,UHF34,Bensonhurst - Bay Ridge,Summer 2018,7.83,2018-06-01,00:00:00
4270,209,1.142849e+08,62243.281229,"POLYGON ((-73.83979 40.83562, -73.83967 40.835...",605827.0,365.0,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,UHF34,Bensonhurst - Bay Ridge,Winter 2017-18,8.10,2017-12-01,00:00:00
4271,209,1.142849e+08,62243.281229,"POLYGON ((-73.83979 40.83562, -73.83967 40.835...",605954.0,365.0,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,Parkchester and Soundview (CD9),Annual Average 2018,7.11,2018-01-01,00:00:00
4272,209,1.142849e+08,62243.281229,"POLYGON ((-73.83979 40.83562, -73.83967 40.835...",605955.0,365.0,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,Parkchester and Soundview (CD9),Summer 2018,8.50,2018-06-01,00:00:00


In [ ]:
gdf_join.dropna()
print(gdf_join.shape[0])

4274


In [ ]:
gdf_join.dtypes

boro_cd              int64
shape_area         float64
shape_leng         float64
geometry          geometry
unique_id          float64
indicator_id       float64
name                object
measure             object
measure_info        object
geo_type_name       object
geo_place_name      object
time_period         object
start_date          object
data_value         float64
message            float64
Date                object
Time                object
dtype: object

In [ ]:
print (gdf_join.crs)
print (gdf_join.crs.name)
print (gdf_join.crs.datum)

epsg:4326
WGS 84
World Geodetic System 1984 ensemble


In [ ]:
gdf_join["center"] = gdf_join["geometry"].centroid
gdf_join.head()

,boro_cd,shape_area,shape_leng,geometry,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_place_name,time_period,start_date,data_value,message,Date,Time,center
0,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179744.0,642.0,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,34.4,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960)
1,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179792.0,640.0,Boiler Emissions- Total SO2 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,1.1,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960)
2,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",179840.0,641.0,Boiler Emissions- Total PM2.5 Emissions,Number per km2,number,UHF42,Borough Park,2015,2015-01-01T00:00:00.000,0.2,NaN,2015-01-01,00:00:00,POINT (-73.88753 40.84960)
3,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325103.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,UHF42,Borough Park,2016,2016-01-01T00:00:00.000,0.6,NaN,2016-01-01,00:00:00,POINT (-73.88753 40.84960)
4,206,42664311.5,35875.711733,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",325151.0,645.0,Traffic Density- Annual Vehicle Miles Traveled...,million miles,per km2,CD,Belmont and East Tremont (CD6),2016,2016-01-01T00:00:00.000,3.8,NaN,2016-01-01,00:00:00,POINT (-73.88753 40.84960)


In [ ]:
gdf_join.drop(columns=['geometry'], axis=1)
gdf_join = gdf_join.rename({'center': 'geometry'}, axis=1)


In [ ]:
#TO BE LOOKED AT --- CHANGING TO PUMA PENDING
gdf_join.head()

ValueError: ignored

**2.NYC Parks Data**

In [ ]:
gdf_parks = gpd.read_file("/content/QoL_NYC_Children/Environmental/Data/Parks/ARCHIVED - Parks Properties.zip")

In [ ]:
gdf_parks.head()

,acquisitio,acres,address,borough,class,commission,communityb,councildis,department,eapply,gisobjid,gispropnum,global_id,jurisdicti,location,mapped,name311,nys_assemb,nys_senate,objectid,omppropid,parentid,permit,permitdist,permitpare,pip_ratabl,precinct,retired,signname,subcategor,typecatego,url,us_congres,waterfront,zipcode,geometry
0,19990222000000.00000,20.907,None,R,PARK,20100106000000.00000,503,51,R-03,Seaside Wildlife Nature Park,100005021.0,R145,{EC14E5C9-9687-49BC-9A7A-77F977DC0448},DPR,"Nelson Ave., Tennyson Dr. and Bulkhead Line",True,Seaside Wildlife Nature Park,64,24,6367.0,R145,R-03,Y,R-03,R-03,Yes,122,False,Seaside Wildlife Nature Park,Neighborhood Park,Neighborhood Park,http://www.nycgovparks.org/parks/R145/,11,Yes,10308,"MULTIPOLYGON (((-74.14227 40.54220, -74.14218 ..."
1,19530514000000.00000,0.061,None,Q,PARK,20090423000000.00000,401,22,Q-01,Strippoli Square,100000375.0,Q355,{62700020-4840-4F4A-A15A-7D65B9A6A794},DPR,"31 Ave., 51 St., 54 St.",True,Strippoli Square,30,12,5916.0,Q355,Q-01,Y,Q-01,Q-01,Yes,114,False,Strippoli Square,Sitting Area/Triangle/Mall,Triangle/Plaza,http://www.nycgovparks.org/parks/Q355/,14,No,11377,"POLYGON ((-73.90749 40.75709, -73.90768 40.757..."
2,19400528000000.00000,1.130,150 34 STREET,B,PARK,20100106000000.00000,307,38,B-07,D'Eemic Playground,100004733.0,B210B,{BFD91324-49C1-46B5-B3E9-E43A989DC40B},DPR,3 Ave. bet. 35 St. and 34 St.,True,D'Emic Playground,51,25,5465.0,B210B,B-07,Y,B-07,B-07,Yes,72,False,D'Emic Playground,Neighborhood Plgd,Playground,http://www.nycgovparks.org/parks/B210B/,7,No,11232,"POLYGON ((-74.00468 40.65584, -74.00492 40.655..."
3,20100517000000.00000,2.160,201/125 BOLTON AVENUE,X,PARK,20100106000000.00000,209,18,X-09,Harding Park,100004033.0,X262,{BDFFC8B5-573A-4771-8E51-601D03705C78},DPR,Bolton Ave. bet. O'Brien Ave. and G St.,False,Harding Park,85,34,4857.0,X262,X-09,Y,X-09,X-09,Yes,43,False,Harding Park,Neighborhood Plgd,Neighborhood Park,http://www.nycgovparks.org/parks/X262/,15,No,10473,"MULTIPOLYGON (((-73.85640 40.80925, -73.85676 ..."
4,19550427000000.00000,1.104,4522 CARPENTER AVENUE,X,PARK,20100106000000.00000,212,11,X-12,Wakefield Playground,100004702.0,X188,{E33C24CE-ACEB-4018-BAF0-3CE447C8A2AF},DPR/DOE,Matilda Ave. to Carpenter Ave. bet. E. 239 St....,False,Wakefield Playground,81,36,6621.0,X188,X-12,Y,X-12,X-12,Yes,47,False,Wakefield Playground,JOP,Jointly Operated Playground,http://www.nycgovparks.org/parks/X188/,16,No,10470,"MULTIPOLYGON (((-73.85416 40.90187, -73.85455 ..."


In [ ]:
gdf_parks.dtypes

acquisitio      object
acres          float64
address         object
borough         object
class           object
commission      object
communityb      object
councildis      object
department      object
eapply          object
gisobjid       float64
gispropnum      object
global_id       object
jurisdicti      object
location        object
mapped          object
name311         object
nys_assemb      object
nys_senate      object
objectid       float64
omppropid       object
parentid        object
permit          object
permitdist      object
permitpare      object
pip_ratabl      object
precinct        object
retired         object
signname        object
subcategor      object
typecatego      object
url             object
us_congres      object
waterfront      object
zipcode         object
geometry      geometry
dtype: object

In [ ]:
gdf_parks.drop_duplicates()
gdf_parks.drop(columns=['acquisitio','class','waterfront','us_congres','url','retired','pip_ratabl','permitpare','permitdist','permit','parentid','nys_senate','nys_assemb','global_id','mapped'])

,acres,address,borough,commission,communityb,councildis,department,eapply,gisobjid,gispropnum,jurisdicti,location,name311,objectid,omppropid,precinct,signname,subcategor,typecatego,zipcode,geometry
0,20.907,None,R,20100106000000.00000,503,51,R-03,Seaside Wildlife Nature Park,100005021.0,R145,DPR,"Nelson Ave., Tennyson Dr. and Bulkhead Line",Seaside Wildlife Nature Park,6367.0,R145,122,Seaside Wildlife Nature Park,Neighborhood Park,Neighborhood Park,10308,"MULTIPOLYGON (((-74.14227 40.54220, -74.14218 ..."
1,0.061,None,Q,20090423000000.00000,401,22,Q-01,Strippoli Square,100000375.0,Q355,DPR,"31 Ave., 51 St., 54 St.",Strippoli Square,5916.0,Q355,114,Strippoli Square,Sitting Area/Triangle/Mall,Triangle/Plaza,11377,"POLYGON ((-73.90749 40.75709, -73.90768 40.757..."
2,1.130,150 34 STREET,B,20100106000000.00000,307,38,B-07,D'Eemic Playground,100004733.0,B210B,DPR,3 Ave. bet. 35 St. and 34 St.,D'Emic Playground,5465.0,B210B,72,D'Emic Playground,Neighborhood Plgd,Playground,11232,"POLYGON ((-74.00468 40.65584, -74.00492 40.655..."
3,2.160,201/125 BOLTON AVENUE,X,20100106000000.00000,209,18,X-09,Harding Park,100004033.0,X262,DPR,Bolton Ave. bet. O'Brien Ave. and G St.,Harding Park,4857.0,X262,43,Harding Park,Neighborhood Plgd,Neighborhood Park,10473,"MULTIPOLYGON (((-73.85640 40.80925, -73.85676 ..."
4,1.104,4522 CARPENTER AVENUE,X,20100106000000.00000,212,11,X-12,Wakefield Playground,100004702.0,X188,DPR/DOE,Matilda Ave. to Carpenter Ave. bet. E. 239 St....,Wakefield Playground,6621.0,X188,47,Wakefield Playground,JOP,Jointly Operated Playground,10470,"MULTIPOLYGON (((-73.85416 40.90187, -73.85455 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010,0.117,219 - 225 WEST 122 STREET,M,20100106000000.00000,110,9,M-10,Joseph Daniel Wilson Garden,100004815.0,M351,DPR,W. 122 St. bet. Fred Douglass Blvd. and Adam C...,Joseph Daniel Wilson Garden,87744.0,M351,28,Joseph Daniel Wilson Garden,Greenthumb,Garden,10027,"POLYGON ((-73.95042 40.80774, -73.95060 40.807..."
2011,0.545,None,M,None,102,1,M-02,Adrienne's Playground,100008287.0,M395,DPR,La Guardia Pl. bet. W. 3 St. and Bleecker St.,Fiorello La Guardia Park,6301.0,M395,6,Fiorello La Guardia Park,Neighborhood Plgd,Neighborhood Park,10012,"POLYGON ((-73.99773 40.72911, -73.99781 40.729..."
2012,0.006,None,Q,20090423000000.00000,402,30,Q-02,Strip,100000456.0,Q360A,DPR,53 Ave.bet. 43 St. and 44 St.,Park,5693.0,Q360A,108,Park,Sitting Area/Triangle/Mall,Strip,11378,"POLYGON ((-73.92393 40.73250, -73.92352 40.732..."
2013,0.064,147 TOMPKINS AVENUE,B,20100106000000.00000,303,36,B-03,All People's Church Of The Apostolic Fa*,100004445.0,B439,DPR,Tompkins Ave. and Willoughby Ave.,All People's Church of the Apostolic Faith Com...,5242.0,B439,79,All People's Church of the Apostolic Faith Com...,Greenthumb,Garden,11206,"POLYGON ((-73.94567 40.69437, -73.94599 40.694..."


In [ ]:
gdf_parks.dropna()
print(gdf_parks.shape[0])

2015


In [ ]:
print (gdf_parks.crs)
print (gdf_parks.crs.name)
print (gdf_parks.crs.datum)

epsg:4326
WGS 84
World Geodetic System 1984 ensemble


In [ ]:
puma_gdf = gpd.read_file("/content/QoL_NYC_Children/Access/Data/PUMA", driver = "shapefile")
puma_gdf.head()


,PUMA,Shape_Leng,Shape_Area,geometry
0,3701,53227.113608,9.792852e+07,"POLYGON ((1015289.353 261467.596, 1015334.054 ..."
1,3702,106167.592328,1.889937e+08,"POLYGON ((1021632.336 267934.439, 1022108.577 ..."
2,3703,305269.138803,2.676436e+08,"MULTIPOLYGON (((1042822.450 243439.814, 104255..."
3,3704,47970.203023,1.062169e+08,"POLYGON ((1025507.170 246350.149, 1025418.562 ..."
4,3705,68697.599919,1.224837e+08,"POLYGON ((1012111.566 240863.078, 1012088.246 ..."
